In [5]:
# !pip install "flaml[automl]"
# !pip install rapidfuzz

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os

In [ ]:
from nq_agents import WorkflowAutogen
from nq_agents import data_extraction

# Initialize the multi-agent system
my_agents = WorkflowAutogen()

# Read input examples
GOLD_FILE = 'data/v1.0-simplified_nq-dev-all_sample40_seed32.jsonl'
num_examples = data_extraction.count_lines(GOLD_FILE)

input_examples = data_extraction.read_first_lines(GOLD_FILE, num_examples)

# Make predictions. side effect: write to file
save_path = GOLD_FILE.replace('.jsonl', '_predictions.jsonl')
predictions = my_agents.predict_batch(examples=input_examples, save_path=save_path, verbose=True)


### Evaluate


In [ ]:
from nq_agents import evaluation

# Evaluate predictions
PREDICTIONS_FILE = save_path # generated by multi_agent.py

result = evaluation.evaluate_predictions(GOLD_FILE, PREDICTIONS_FILE)

In [ ]:
result